# Base: Imports and Cleaning

In [11]:
import pandas as pd
import numpy as np
import re

In [4]:
data = pd.read_csv("../World_Datasets/final_dataset_world.csv")

In [5]:
data

,Country,S&P Rating,Country Name,country_code,unemployment_2000,unemployment_2014,unemployment_2015,unemployment_2016,unemployment_2017,unemployment_2018,...,total_reserves_2016,total_reserves_2017,total_reserves_2018,total_reserves_2019,total_reserves_2020,total_reserves_2021,total_reserves_2022,total_reserves_2023,Unnamed: 73,value_counts
0,Australia,AAA,Australia,AUS,6.288000,6.078000,6.055000,5.711000,5.592000,5.300000,...,5.248079e+10,6.565372e+10,5.390953e+10,5.799469e+10,4.254463e+10,5.787750e+10,5.670190e+10,6.170335e+10,3.085167e+10,73.0
1,Canada,AAA,Canada,CAN,6.829000,7.023000,6.945000,7.038000,6.426000,5.837000,...,8.271811e+10,8.667771e+10,8.392560e+10,8.529711e+10,9.042814e+10,1.066151e+11,1.069524e+11,1.175509e+11,5.877545e+10,73.0
2,Denmark,AAA,Denmark,DNK,4.476000,6.925000,6.278000,5.989000,5.833000,5.131000,...,6.421581e+10,7.524445e+10,7.094208e+10,6.683555e+10,7.282335e+10,8.223584e+10,9.607255e+10,1.093708e+11,5.468542e+10,73.0
3,Germany,AAA,Germany,DEU,7.917000,4.981000,4.624000,4.122000,3.746000,3.384000,...,1.840313e+11,1.999831e+11,1.980271e+11,2.240280e+11,2.684086e+11,2.957362e+11,2.939137e+11,3.227001e+11,1.613500e+11,62.0
4,Liechtenstein,AAA,Liechtenstein,LIE,9.640537,9.136085,9.052059,8.903803,8.664479,8.383856,...,2.453835e+09,2.181187e+09,1.908538e+09,1.635890e+09,1.363242e+09,1.090593e+09,8.179449e+08,5.452966e+08,2.726483e+08,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,Sri Lanka,SD,Sri Lanka,LKA,7.740000,4.157000,4.519000,4.243000,4.046000,4.318000,...,6.008199e+09,7.959048e+09,6.920826e+09,7.648305e+09,5.663994e+09,3.136992e+09,2.352744e+09,1.568496e+09,7.842479e+08,71.0
127,Ukraine,SD,Ukraine,UKR,11.707000,9.270000,9.140000,9.350000,9.500000,8.799000,...,1.553726e+10,1.881093e+10,2.081790e+10,2.531700e+10,2.913754e+10,3.096667e+10,2.850593e+10,4.051011e+10,2.025505e+10,67.0
128,Lebanon,D,Lebanon,LBN,8.594000,8.796000,9.270000,9.760000,10.236000,10.741000,...,5.390551e+10,5.541153e+10,5.238061e+10,5.221348e+10,4.244040e+10,3.523922e+10,3.251288e+10,2.167525e+10,1.083763e+10,66.0
129,Puerto Rico,D,Puerto Rico,PRI,10.080000,13.900000,12.000000,11.800000,10.800000,9.200000,...,4.084615e+10,3.630769e+10,3.176923e+10,2.723077e+10,2.269231e+10,1.815385e+10,1.361538e+10,9.076923e+09,4.538462e+09,39.0


In [8]:
print('Number of unique values: ', data['S&P Rating'].nunique())
print(data['S&P Rating'].unique())

Number of unique values:  38
['AAA' 'AA+' 'AA+\xa0' 'AA-' 'AA' 'AA\xa0' 'A+' 'NR' 'A\xa0' 'A' 'AA-\xa0'
 'A-' 'A-\xa0' 'BBB+' 'BBB' 'BBB\xa0' 'BBB+\xa0' 'BBB-' 'BBB-\xa0'
 'BB+\xa0' 'BB+' 'BB\xa0' 'BB' 'BB-\xa0' 'BB-' 'B+' 'B+\xa0' 'SD' 'B\xa0'
 'B-' 'B' 'B-\xa0' 'CCC+\xa0' 'CCC+' 'CCC' 'D' 'D\xa0' 'SD\xa0']


As 38 is too many values for a classification, we will regroup first, and drop the non-rated 'NR'.

In [9]:
data = data[data['S&P Rating'] != 'NR'] #dropping non-rated

In [13]:
data['target'] = np.where(data['S&P Rating'].str.match(r'.*A.*', na=False), 0, 
                          np.where(data['S&P Rating'].str.match(r'.*B.*', na=False), 1, 2))

C:\Users\ginny\AppData\Local\Temp\ipykernel_11856\2029520729.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['target'] = np.where(data['S&P Rating'].str.match(r'.*A.*', na=False), 0,


We now have 3 classification possibilities. 

# Base Model